In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [16]:
DELETE_TABLES=True
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%osm_intersections_table_name
        cursor.execute(cmd)
    except Exception as e:
        print(e)
        pass
    cmd="""DROP TABLE `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)
2

2

In [17]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `contour_line_id` INT NOT NULL,
    `elevation` FLOAT,

    FOREIGN KEY (contour_line_id) REFERENCES %s(id),
    PRIMARY KEY (osm_id)
    );
        """%(osm_nodes_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)



    
    cmd="""CREATE TABLE `%s`(
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `contour_line_id` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
        FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
        FOREIGN KEY (contour_line_id) REFERENCES %s(id),
        PRIMARY KEY (osm_begin,osm_end,osm_key,edge_coordinate)
        

          )
          ENGINE=InnoDB;
        """%(osm_intersections_table_name,osm_nodes_table_name,osm_nodes_table_name,contours_lines_table_name)
    cursor.execute(cmd)
    

2

2

In [5]:
t1=time.time()
G=build_graph(cursor)
max_depth=np.max([data['depth'] for _,data in G.nodes(data=True)])
t2=time.time()
print(t2-t1)

0.940648078918457


In [6]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

67.59827899932861


(17435, 20792)

In [7]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry','length']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
osm_edges_df=osm_edges_df.sort_values(by='length')
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
t2=time.time()
print(t2-t1)
osm_edges_df

0.40482306480407715


,edge,geometry,length,number_of_points
0,"(3228062295, 4575394776, 0)","LINESTRING (536149.291 4707866.081, 536149.100...",1.140,2
1,"(8322586073, 8322586072, 0)","LINESTRING (535461.183 4720987.574, 535462.311...",1.410,2
2,"(9015392280, 9022366875, 0)","LINESTRING (479087.515 4643413.911, 479086.898...",1.478,2
3,"(1798092110, 10807186012, 0)","LINESTRING (535396.472 4721724.923, 535397.887...",1.821,2
4,"(10758881318, 344189032, 0)","LINESTRING (516965.588 4701096.308, 516967.429...",1.852,2
...,...,...,...,...
20787,"(498564206, 498565647, 0)","LINESTRING (497333.270 4598859.449, 497336.639...",19370.256,602
20788,"(1373773448, 3102090356, 0)","LINESTRING (515361.514 4646505.379, 515333.106...",20568.484,850
20789,"(60608128, 297444509, 0)","LINESTRING (470534.042 4685576.155, 470528.651...",23348.990,1069
20790,"(283719550, 498407678, 0)","LINESTRING (485161.514 4677932.541, 485167.944...",29491.401,1304


In [8]:
t1=time.time()
intersection,depth=decreasing_depth_intersections(osm_edges_df,osm_crs,G,cursor,max_depth,depth_step=20)
t2=time.time()
print('total : %f'%(t2-t1),depth)

520<depth<=540 :0.066226
500<depth<=520 :0.128716
480<depth<=500 :0.180478
460<depth<=480 :0.304476
440<depth<=460 :0.434397
420<depth<=440 :0.728540
400<depth<=420 :1.180438
380<depth<=400 :1.709708
360<depth<=380 :1.962698
340<depth<=360 :2.442300
320<depth<=340 :2.772994
300<depth<=320 :3.449058
280<depth<=300 :4.102990
260<depth<=280 :5.365866
240<depth<=260 :5.489183
220<depth<=240 :7.324142
200<depth<=220 :8.362483
180<depth<=200 :10.420251
160<depth<=180 :13.679251
140<depth<=160 :13.238813
120<depth<=140 :15.587888
100<depth<=120 :18.483980
80<depth<=100 :24.498045
60<depth<=80 :29.596426
40<depth<=60 :26.732352
20<depth<=40 :30.859074
0<depth<=20 :45.343333
total : 274.449279 1


In [9]:
intersection['edge_coordinate']=None
t1=time.time()
for edge,df in intersection.groupby('edge'):
    ls=osm_edges_df[osm_edges_df.edge==edge].iloc[0]['geometry']
    df['edge_coordinate']=df['geometry'].apply(lambda pt:ls.project(pt))
    intersection.loc[intersection.edge==edge,'edge_coordinate']=df['edge_coordinate']
t2=time.time()
print(t2-t1)

109.16170978546143


In [10]:
t1=time.time()
pre_osm_nodes_to_contours={}
for edge,df in intersection.groupby('edge'):
    df=df.sort_values('edge_coordinate')
    u,v=edge[:2]
    if not u in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[u]={df.iloc[0]['id']}
    else:
        pre_osm_nodes_to_contours[u].add(df.iloc[0]['id'])
    if not v in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[v]={df.iloc[-1]['id']}
    else:
        pre_osm_nodes_to_contours[v].add(df.iloc[-1]['id'])
t2=time.time()
print(t2-t1)

18.351147174835205


In [11]:
t1=time.time()
osm_nodes_to_contours={}
ambiguous_osm_nodes_to_contours={}
for osm_node,contour_ids in pre_osm_nodes_to_contours.items():
    if len(contour_ids)>1:
        id_1,id_2=list(contour_ids)[:2]
        if id_2 in G.neighbors(id_1):
            osm_nodes_to_contours[osm_node]=id_1
        else:
            predecessors=list(G.predecessors(id_2))
            contour_id=id_2 if len(predecessors)==0 else predecessors[0]
            osm_nodes_to_contours[osm_node]=contour_id
    else:
        ambiguous_osm_nodes_to_contours[osm_node]=list(contour_ids)[0]
t2=time.time()
print(t2-t1)

0.028282880783081055


In [13]:
t1=time.time()
ambiguous_contours_df=get_nodes_data(cursor,set(ambiguous_osm_nodes_to_contours.values()))
ambiguous_contours_df=ambiguous_contours_df.set_index('id')
ambiguous_contours_df=ambiguous_contours_df.to_crs(osm_crs)
ambiguous_contours_df['geometry']=ambiguous_contours_df['geometry'].apply(lambda ls:Polygon(ls))
t2=time.time()
print('df ready, took %f'%(t2-t1))
for osm_node,id_contour in ambiguous_osm_nodes_to_contours.items():
    datum=G_osm.nodes()[osm_node]
    pt=Point(datum['x'],datum['y'])
    if pt.within(ambiguous_contours_df.loc[id_contour]['geometry']):
        osm_nodes_to_contours[osm_node]=id_contour
    else:
        predecessors=list(G.predecessors(id_contour))
        id_contour=id_contour if len(predecessors)==0 else predecessors[0]
        osm_nodes_to_contours[osm_node]=id_contour

t3=time.time()
print('ambiguities removed, took %f'%(t3-t2))

df ready, took 86.377027
ambiguities removed, took 202.619630


In [18]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    t1=time.time()
    string_list=',\n'.join([str((u,v)) for u,v in osm_nodes_to_contours.items()])
    cmd="INSERT INTO %s (`osm_id`,`contour_line_id`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    string_list=',\n'.join(['(%i,%i,%i,%i,%f,%f)'%(row['id'],row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'])  for _,row in intersection.iterrows()])

    cmd="INSERT INTO %s (`contour_line_id`,`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`) VALUES %s;"%(osm_intersections_table_name,string_list)
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)

8.75727105140686
